In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re 
import string

In [39]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [40]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [41]:
df_true.head(5)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [42]:
df_fake["class"] = 0
df_true["class"] = 1

In [43]:
df_fake.shape, df_true.shape


((23481, 5), (21418, 5))

In [44]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [45]:
df_merge.columns


Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [46]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [47]:
df.isnull().sum()


text     0
class    0
dtype: int64

In [48]:
df = df.sample(frac = 1)


In [49]:
df.head()

,text,class
13774,Barack only needs to find the closest mirror t...,0
17585,LONDON (Reuters) - British Prime Minister Ther...,1
15941,"LONDON (Reuters) - Arron Banks, a tycoon who b...",1
13657,Facebook CEO Mark Zuckerberg appears to be con...,0
18103,"ERBIL, Iraq (Reuters) - Jalal Talabani, whose ...",1


In [50]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [51]:
df.columns

Index(['text', 'class'], dtype='object')

In [52]:
df.head()

,text,class
0,Barack only needs to find the closest mirror t...,0
1,LONDON (Reuters) - British Prime Minister Ther...,1
2,"LONDON (Reuters) - Arron Banks, a tycoon who b...",1
3,Facebook CEO Mark Zuckerberg appears to be con...,0
4,"ERBIL, Iraq (Reuters) - Jalal Talabani, whose ...",1


In [53]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [54]:
df["text"] = df["text"].apply(wordopt)

In [55]:
x = df["text"]
y = df["class"]

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [58]:
from sklearn.linear_model import PassiveAggressiveClassifier
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(xv_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [59]:
pred_pr=classifier.predict(xv_test)


In [60]:
classifier.score(xv_test,y_test)

0.9949220489977728

In [61]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(xv_train,y_train)

MultinomialNB()

In [62]:
pred_mnb = mnb.predict(xv_test)
mnb.score(xv_test,y_test)

0.9338084632516703

In [63]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_PC = classifier.predict(new_xv_test)
    pred_MNB = mnb.predict(new_xv_test)
    

    return print("\n\nPassive Aggressive Classifier Prediction: {}\nMultinomial Prediction: {}".format(output_lable(pred_PC[0]),output_lable(pred_MNB[0])))
                                                                                                              

In [64]:
news = str(input())
manual_testing(news)




Passive Aggressive Classifier Prediction: Fake News
Multinomial Prediction: Not A Fake News
